### Simple SPO framework for CSC

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import utils

DATA_FILE = "../data-raw/sample.csv"

data = pd.read_csv(DATA_FILE, header = None)
data = data[0:10] # take a slice
n_sample = data.shape[0]
n_asset = data.shape[1]

In [6]:
# writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
#  ... Session context
# writer.close()

rt = tf.placeholder(name="returns", dtype=tf.float64, shape=data.shape) # tradable assets
w = tf.Variable(tf.zeros(shape=(data.shape[1], 1), dtype=tf.float64), name='weights', dtype=tf.float64)
l = tf.Variable(tf.zeros(shape=(1, 1), dtype=tf.float64), name='lambda', dtype=tf.float64) # lambda for self-financing

# single variable constraints
w = tf.assign(w, tf.clip_by_value(w, 0.5, 0))
# w = tf.get_variable(name='weights', shape=data.shape, initializer=tf.zeros_initializer())
# l = tf.get_variable(name='lambda', shape=(1,1), initializer=tf.zeros_initializer())

rp = tf.reduce_mean(tf.tensordot(rt, w[0:data.shape[1]], 1)) # alt: tf.nn.moments(rt, axes=[1])
err = tf.tensordot((rt - tf.reduce_mean(rt, 0)), w, 1)
risk = tf.tensordot(tf.transpose(err), err, 1) / n_sample # E w^T (X - EX)^T (X - EX) w

sf_const = 1 - tf.reduce_sum(w) # self-financing constraints, Lagragian Duality

obj = - rp + 50 * risk + l * sf_const
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(obj)

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(200):
        sess.run(optimizer, feed_dict={rt: data})
    print(sess.run(risk, feed_dict={rt: data}))

[[0.00064571]]
